In [10]:
import numpy as np
import pylab as pl 
import matplotlib.pyplot as plt
%matplotlib inline

# Neural Networks

In [11]:
class Neuralnets(object):
    
    def __init__(self,classes):
        self.classes = classes
        
    
    # Initialize the weights as a numpy array (All weights will be matrices)
    # Initialize the bias terms as a numpy array as well (Bias terms will be vectors)
    
    def buildNeuralNetworks(self,X,y,hL,nodes,iteration,eta):
        # SGD
        parameters = self.ParameterInitialization(X,y,nodes)
        weightStorage = parameters[0]
        biasStorage = parameters[1]
        for i in xrange(0,iteration):
            np.random.shuffle(X)
            samples,featuress = X.shape
            for j in xrange(0,samples):
                x = X[j,]
            
                # Feed Forward
                # Store W,a,f'(z)
                #weightStorage = np.empty()
                valueStorage = []
                activeDerivStorage = []
                #a = x
                valueStorage.append(x)
                #print x
                for k in range(hL+1):
                    #print k
                    #print a
                    #print weightStorage[k]
                    #print valueStorage[k]
                    #print biasStorage[k]
                    z =np.dot(np.transpose(weightStorage[k]),valueStorage[k]) +biasStorage[k]
                    #print '--z-value--'
                    #print z
                    if k != hL:
                        #print z
                        f = np.vectorize(self.ReLU)
                        a = f(z)
                        
                        #print '--a-value--'
                        #print a.shape
                        #print a
                        
                        valueStorage.append(a)
                        #print valueStorage.shape
                        fPrime = np.vectorize(self.ReLUDeriv)
                        activeDerivStorage.append(fPrime(z))
                    else:
                        #f = np.vectorize(self.softmax)
                        a = self.softmax(z)
                        #print '--a-value--'
                        #print a.shape
                        #print a
                        valueStorage.append(a)
                        #fPrime = np.vectorize(self.softmaxDeriv)
                        activeDerivStorage.append(self.softmaxDeriv(z))
                
                #print y[j]
                #print valueStorage[hL+1]
                #print valueStorage
                #print len(valueStorage)
                
                #lossDeriv = self.crossEntropyLoss(y[j],valueStorage[hL+1]) # Use the value from the output layer (last)
                
                # diagonal of the softmax matrix to calculate the output error
                # Use the entire matrix of softmax Jacobian for this purpose
                #print lossDeriv
                #print z
                
                #outputError = np.dot(self.softmaxDeriv(z),np.transpose(lossDeriv))
                outputError =  valueStorage[hL+1] - y[j]
                #print outputError
                #print outputError
                #print np.dot(self.softmaxDeriv(z),np.transpose(lossDeriv))
                #print self.softmaxDeriv(z)
                
                #print np.transpose(lossDeriv)
                #print outputError
                
                #Backpropagation 
                # Store delta at each iteration
                deltaStorage = [None]*(hL+1)
                deltaStorage[hL] = outputError
                for l in reversed(range(hL)):
                    #print l
                    #layer = hL-l
                    #print layer
                    #print np.diag(activeDerivStorage[l])
                    
                    #print weightStorage[l]
                    #print np.diag(activeDerivStorage[l])
                    #print deltaStorage[l+1]
                    delta = np.dot(np.dot(np.diag(activeDerivStorage[l]),weightStorage[l+1]),deltaStorage[l+1])
                    deltaStorage[l] = np.array(delta)
                    #print deltaStorage
                 
                
                for k in range(1,hL+1):
                    weightGradient = np.outer(valueStorage[k],deltaStorage[k])
                    biasGradient = deltaStorage[k]
                    weightStorage[k] = weightStorage[k] - (eta*weightGradient)
                    #print '--weight Matrices--'
                    #print weightStorage[k]
                    biasStorage[k] = biasStorage[k] - (eta*biasGradient)
                    #print '--bias Vectors--'
                    #print biasStorage[k]
                    
                #print biasStorage,weightStorage
        return (weightStorage,biasStorage)
                    
    
    
    def predictNN(self,X,weight,Bias):
        layers = len(weight)
        inputValue = X
        hiddenLayers = layers - 1
        valueVector = []
        for i in range(layers):
            if i == 0:
                newLayerValue = np.dot(inputValue,weight[i])+Bias[i]
                f = np.vectorize(self.ReLU)
                valueVector.append(f(newLayerValue))
            else:
                newLayerValue = np.dot(valueVector[i-1],weight[i])+Bias[i]
                f = np.vectorize(self.ReLU)
                valueVector.append(f(newLayerValue))
                
        #print valueVector        
        #print valueVector[-1] # Get the last value in the value vector i.e. get the value from the last hidden layer
        predProb = np.apply_along_axis(self.softmax, 1, valueVector[-1])
        
        #print valueVector[-1]
        #print predProb
        u = np.argmax(predProb,1) # get the element with the highest probability
        pred = np.zeros((X.shape[0],predProb.shape[1]))
        #pred
        pred[np.arange(len(u)), u] = 1
        return (pred)
        
    def ClassificationError(self,pred,y):
        """ Provides the misclassification error given the true value and the prediction"""
        cError = 1 - (np.count_nonzero(pred*y)/np.float(pred.shape[0]))
        return(cError)
                
    def ParameterInitialization(self,X,y,nodes):
        # Control the number of nodes in the weight initialization function
        # nodes variable should be a list of number of nodes in each hidden layer
        weightList = []
        biasList = []
        allNodes = [X.shape[1]]+nodes+[y.shape[1]]
        
        for i in range(1,len(allNodes)):
            weight = np.empty([allNodes[i-1],allNodes[i]])
            # Gaussian random weights with variance 1/sqrt(m)
            weight = np.reshape(np.random.normal(0, 1/np.sqrt(allNodes[i-1]),weight.size), (allNodes[i-1],allNodes[i]))
            weightList.append(weight)
            
            bias = np.random.normal(0,1/np.sqrt((allNodes[i])),allNodes[i])
            biasList.append(bias)
        return(weightList,biasList)
            
    
    
    def ReLU(self,z):
        return np.amax([0,z])
    
    def softmax(self,Z):
            #denom = sum(np.exp(Z))
        #return(np.exp(Z)/denom)
        e_x = np.exp(Z - np.max(Z))
        return e_x / e_x.sum(axis=0)

    
    def ReLUDeriv(self,z):
        if z>0:
            Derive = 1
        else:
            # subgradient at zero was set to zero
            Derive = 0
        return(Derive)
    
    def softmaxDeriv(self,Z):
        n = len(Z)
        denom = sum(np.exp(Z))
        
        Jacobian = np.empty([n,n])
        for i in range(n):
            for j in range(i+1):
                if i == j:
                    Jacobian[i,j] = (np.exp(Z[i])/denom) - (np.exp(Z[i])/denom)**2
                    Jacobian[j,i] = (np.exp(Z[i])/denom) - (np.exp(Z[i])/denom)**2
                else:
                    Jacobian[i,j] = (np.exp(-(Z[i]+Z[j]))/denom)**2
                    Jacobian[j,i] = (np.exp(-(Z[i]+Z[j]))/denom)**2
        return (Jacobian)
                
    def crossEntropyLoss(self, y,guess):
        return(-(y/guess))
        
    

In [12]:
NNexercise = Neuralnets(2)
a = np.array([[2,3,4.5],[3,14.5,7]])
#a = np.array([[2,3],[3,14.5],[4.5,7]])
NNexercise.softmax(a)
#print a
np.apply_along_axis(NNexercise.softmax, 1, a)
NNexercise.softmax(np.array([1000,40000,1]))
NNexercise.crossEntropyLoss(np.array([1,1,0]),np.array([0.34,0.25,0.41]))
#f = np.vectorize(NNexercise.ReLU)
#f(([-9.6517,0.6259,0.2598]))


array([-2.94117647, -4.        , -0.        ])

In [13]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)
softmax(np.array([1000,40000,1]))

array([ 0.,  1.,  0.])

In [15]:

a = np.array([2.0,3.4,5.6,3.4,-0.1,0,-9])
NNexercise.softmaxDeriv(a)
NNexercise.ReLU(a.all)
b = np.array([[2.0,3.4],[5.6,3.4],[-0.1,0],[-9,0.1]])
f = np.vectorize(NNexercise.ReLU)  # or use a different name if you want to keep the original f

#result_array = f(b) 
#result_array
X = np.array([[-0.1,0],[-9,2]])
y = np.array([[1,0,0],[0,1,0]])
nodes = [4]
Params = NNexercise.ParameterInitialization(X,y,nodes)
#print Params[1]
Weight,bias = NNexercise.buildNeuralNetworks(X,y,1,nodes,1000,0.01)
#print Weight

In [16]:
pred = NNexercise.predictNN(X,Weight,bias)
print pred
#print y
#pred-y
#np.count_nonzero(pred*y)

[[ 0.  1.  0.]
 [ 1.  0.  0.]]


In [17]:
NNexercise.ClassificationError(pred,y)

1.0

In [18]:
y2 = np.array([[1,0,0],[1,0,0],[0,1,0],[0,1,0]])
pred2 = np.array([[0,1,0],[1,0,0],[0,1,0],[1,0,0]])

In [56]:
NNexercise.ClassificationError(pred2,y2)

0.5

In [11]:
sum(np.exp(a))

n = len(a)
Z = a
denom = sum(np.exp(Z))
        
Jacobian = np.empty([n,n])
for i in range(n):
    for j in range(i+1):
        #print i,j
        if i == j:
            #print (np.exp(Z[i])/denom)
            #print (np.exp(Z[i])/denom)**2
            Jacobian[i,j] = (np.exp(Z[i])/denom) - (np.exp(Z[i])/denom)**2
            Jacobian[j,i] = (np.exp(Z[i])/denom) - (np.exp(Z[i])/denom)**2
        else:
            #print (np.exp(Z[i])/denom)
            #print (np.exp(Z[i])/denom)**2
            Jacobian[i,j] = (np.exp(-(Z[i]+Z[j]))/denom)**2
            Jacobian[j,i] = (np.exp(-(Z[i]+Z[j]))/denom)**2


#print Jacobian

### Using the given data_3class.csv

In [19]:
train = pl.loadtxt('../DataFiles/HW3/data_3class.csv')


X = train[:, 0:2]
Y = train[:, 2:3]
Y
len(Y)
Y_encode = np.int_(np.reshape(Y,(800,)))
#print Y_encodhttp://localhost:8888/notebooks/Documents/Class-6/PythonFiles/HW3.ipynb#

#plt.scatter(X[:, 0], X[:, 1], c=Y, edgecolors='k')

In [20]:
b = np.zeros((len(Y_encode), 3))
b[np.arange(len(Y_encode)), Y_encode] = 1
b
X_sub = np.array(X[352:353,:])
b_sub = b[352]
weightStorage = [np.array([[1,1,1],[1,1,1]]),np.array([[1,1,1],[1,1,1],[1,1,1]])] 
biasStorage = [np.array([-1.19710135,  1.170064  , -0.60924974]),np.array([-0.97939498,  0.70415475,  1.04934362])]

weightStorage = [np.array([[ 0.73176929,  0.00782698, -0.46970402],
       [ 1.48885619, -0.82267122,  1.14324093]]), np.array([[ 0.49598047,  0.06429676, -0.33003664],
       [-1.36422687,  0.49123912,  0.838899  ],
      [ 0.03411018,  1.14214609,  0.84216352]])]
biasStorage2 = [np.array([[-1.19710135,  1.170064  , -0.60924974]]), np.array([[-0.97939498,  0.70415475,  1.04934362]])]
print weightStorage
print biasStorage
print biasStorage2
print X_sub
print b_sub

[array([[ 0.73176929,  0.00782698, -0.46970402],
       [ 1.48885619, -0.82267122,  1.14324093]]), array([[ 0.49598047,  0.06429676, -0.33003664],
       [-1.36422687,  0.49123912,  0.838899  ],
       [ 0.03411018,  1.14214609,  0.84216352]])]
[array([-1.19710135,  1.170064  , -0.60924974]), array([-0.97939498,  0.70415475,  1.04934362])]
[array([[-1.19710135,  1.170064  , -0.60924974]]), array([[-0.97939498,  0.70415475,  1.04934362]])]
[[-1.64878189  1.57572657]]
[ 0.  0.  1.]


In [39]:
NNexercise_data3 = Neuralnets(3)
nodes = [3]
Weight,bias = NNexercise_data3.buildNeuralNetworks(X,b,1,nodes,700,0.0001)

In [31]:
#NNexercise_data3 = Neuralnets(3)
#nodes = [3]
#Params = NNexercise_data3.ParameterInitialization(X,b,nodes)
#print Params[0]
#NNexercise_data3.buildNeuralNetworks(X_sub,b_sub,1,nodes,1,0.0001)

In [32]:
NNexercise_data3 = Neuralnets(3)
nodes = [3]
Params = NNexercise_data3.ParameterInitialization(X,b,nodes)
#print Params[0]
#Weight,bias = NNexercise_data3.buildNeuralNetworks(X,b,1,nodes,10,0.0001)
#print bias

In [38]:
Weight

[array([[-0.78669423,  0.70366009,  1.69770682,  0.11064913],
        [ 1.39517836,  0.59081608,  0.07974418, -0.18270584]]),
 array([[ 0.32668705,  0.46909253, -0.82734596],
        [ 0.71095772,  0.67091148, -0.91517809],
        [-0.66544632,  0.73399975, -0.76801792],
        [-0.15608515,  0.55759189,  1.12110288]])]

In [34]:
pred = NNexercise.predictNN(X,Weight,bias)
#pred

In [35]:
pred

array([[ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       ..., 
       [ 1.,  0.,  0.],
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.]])

In [36]:
NNexercise.ClassificationError(pred,b)

0.6775

## Trial Runs

In [ ]:
One small  hidden layer - 3
One large hidden layer - 50
Two small hidden layer - [3,4]
Two large hidden layer - [50,60]

# 2D Datasets

In [45]:
data = pl.loadtxt('../DataFiles/HW2/data1_train.csv')
len(data[:,2])
Y = data[:,2]
Y = np.int_(np.reshape(Y,(len(Y,))))
X = data[:,0:2]
b = np.zeros((len(Y), 2))
b[np.arange(len(Y)), Y] = 1
b

array([[ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,

array([ 1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1., -1.,
       -1.,  1., -1.,  1.,  1., -1., -1.,  1.,  1.,  1., -1.,  1.,  1.,
        1., -1., -1., -1.,  1.,  1.,  1., -1., -1.,  1.,  1., -1., -1.,
       -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,
       -1.,  1.,  1.,  1., -1.,  1., -1., -1.,  1.,  1.,  1., -1., -1.,
       -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1., -1., -1.,
        1.,  1.,  1.,  1.,  1.,  1., -1., -1.,  1., -1., -1.,  1., -1.,
       -1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1., -1.,  1.,  1., -1.,
       -1.,  1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1., -1., -1.,  1.,
       -1.,  1., -1., -1.,  1., -1., -1., -1., -1., -1.,  1.,  1.,  1.,
        1., -1., -1., -1.,  1., -1., -1.,  1., -1.,  1., -1.,  1., -1.,
       -1., -1., -1., -1., -1.,  1., -1., -1.,  1.,  1., -1.,  1.,  1.,
        1.,  1.,  1., -1.,  1.,  1., -1.,  1., -1., -1., -1.,  1., -1.,
       -1., -1.,  1., -1., -1.,  1.,  1., -1., -1.,  1.,  1., -1

In [46]:
# MNIST dataset
# 10 class digit classification
# Atleast 200 training samples, validation and testing to be 150 samples

# Convolutional Neural Networks

In [142]:
import tensorflow as tf

# Scratch

In [23]:
a = np.random.rand(3,3)
b = np.random.rand(3,2)
c = np.random.rand(4,3)
np.exp(c)

array([[ 1.46888017,  2.1098551 ,  1.32734838],
       [ 1.86130614,  1.91456954,  2.64912421],
       [ 2.47338498,  1.23116249,  1.17661799],
       [ 1.57488247,  2.20459446,  1.87048811]])

In [21]:
bigArray = [[a],[b],[c]]

In [22]:
bigArray[2]

[array([[ 0.66995296,  0.23646859,  0.53870775],
        [ 0.37648099,  0.56302431,  0.48723579],
        [ 0.47250215,  0.76336075,  0.20623173],
        [ 0.21332375,  0.99115811,  0.85635214]])]

In [3]:
for j in range(10,20):
    print 20-j

10
9
8
7
6
5
4
3
2
1


In [30]:
a = np.array([2,3.3,9,2.2])
b = np.array([2.9,7.3,1.9,3.2])
a/b

array([ 0.68965517,  0.45205479,  4.73684211,  0.6875    ])

In [7]:
np.exp(a)/sum(np.exp(a))

array([  9.07009851e-04,   3.32808822e-03,   9.94657078e-01,
         1.10782433e-03])

In [8]:
sum(np.exp(a)/sum(np.exp(a)))


1.0

In [44]:
W = np.empty()
for a in range(5):
    W[a] = np.rand(2,a)

TypeError: Required argument 'shape' (pos 1) not found

In [51]:
W = []
A = np.array([[2.9,7.3],[1.9,3.2]])
B = np.array([[2.9,7.3,23],[1.9,3.2,8]])
W.append(A)
W.append(B)

In [57]:
np.log(W[1])

array([[ 1.06471074,  1.98787435,  3.13549422],
       [ 0.64185389,  1.16315081,  2.07944154]])

In [58]:
np.array(1,[2,3])

TypeError: data type not understood

In [60]:
a = np.empty([2,3])
a.fill(1/np.sqrt(3))
a

array([[ 0.57735027,  0.57735027,  0.57735027],
       [ 0.57735027,  0.57735027,  0.57735027]])

In [72]:
a = 2
s = [3]
u = [4]

b = s+[a]+u
b

[3, 2, 4]

In [108]:
b

array([[ 2. ,  3.4],
       [ 5.6,  3.4],
       [-0.1,  0. ],
       [-9. ,  0.1]])

In [127]:
A = [None]*8
for i in range(8):
    A[i]=i**3
A

[0, 1, 8, 27, 64, 125, 216, 343]

In [130]:
A = [None]*8
A[8]

IndexError: list index out of range

In [143]:
W

[array([[ 2.9,  7.3],
        [ 1.9,  3.2]]), array([[  2.9,   7.3,  23. ],
        [  1.9,   3.2,   8. ]])]

In [198]:
A = [i**2 for i in range(10)]
for i in reversed(range(10)):
    print A[i]

81
64
49
36
25
16
9
4
1
0


In [58]:
a = np.array([2,3,4])
b = np.array([2,43])
print a.shape
print b.shape
A = np.outer(a,b)
A.size

(3,)
(2,)


6

In [59]:
np.random.normal(0, 1/np.sqrt(2), 10)

array([ 0.83526238, -0.40728819, -0.10133286, -0.01758712,  0.22786795,
       -0.0959355 ,  0.43493825,  1.44427668, -0.5360688 ,  0.09329915])

In [60]:
np.array((np.random.normal(0, 1/np.sqrt(7), A.size)),size=(3,2))

TypeError: 'size' is an invalid keyword argument for this function

In [61]:
A = np.reshape(np.random.normal(0, 1/np.sqrt(7), A.size),(4,3))
A

ValueError: total size of new array must be unchanged

In [62]:
u = np.argmax(A,1)

In [63]:
pred = np.zeros((3,3))
pred
pred[np.arange(3), u] = 1

In [67]:
pred[np.arange(3), u] = 1
pred

array([[ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.]])

In [70]:
pred  * pred

array([[ 0.,  1.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  1.,  0.]])

In [309]:
a = [2,3,4,5]
a[-1]

5

In [67]:
X

array([[-9. ,  2. ],
       [ 5.6,  3.4],
       [ 2. ,  3.4],
       [-0.1,  0. ]])

In [68]:
np.random.shuffle(X)

In [69]:
X

array([[-9. ,  2. ],
       [-0.1,  0. ],
       [ 2. ,  3.4],
       [ 5.6,  3.4]])

In [24]:
layers = 3
for i in reversed(range(1,layers-1)):
    print i

1


In [25]:
for i in range(1,layers):
    print i

1
2


In [150]:
A = [2,3,4]


In [247]:
A [2] - 0.5 * A [2]
q = np.array([0,9,-2,901])
q

array([  0,   9,  -2, 901])

In [252]:
np.amax([0,-9])

0